In [9]:
pip install neo4j-driver

Note: you may need to restart the kernel to use updated packages.


In [10]:
from neo4j import GraphDatabase, basic_auth
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

db_location = "bolt://neo4j:7687"
username = "neo4j"
password = "qwerty"

query = """
        MATCH (p:Person)-[sm:SMOKES]->(t:Tobacco), (p)-[lv:LIVES]->(s:State)
        RETURN s.name as state_name, p.gender as gender, p.age as age, t.name as tobacco
        """


def get_dataset(tx):
    db_res = tx.run(query)
    training_data = pd.DataFrame([r.values() for r in db_res], columns=db_res.keys())
    return training_data

In [11]:
db = GraphDatabase.driver(db_location, auth=basic_auth(username, password))
fe = OrdinalEncoder()
multilinear_model = MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=42))

with db.session() as session:
    df = session.read_transaction(get_dataset)

df.sample(frac=1).head()

,state_name,gender,age,tobacco
5302,Kansas,Male,18,Smokeless Tobacco
1438,Texas,Male,28,Cigarette
1286,Texas,Male,27,Cigarette
6806,Kentucky,Male,33,Cigarette
5509,Kansas,Female,25,Cigarette


In [12]:
Y_data = np.column_stack((df['age'].values, fe.fit_transform(df[['gender']])))
X_data = fe.fit_transform(df[['state_name', 'tobacco']])

pd.DataFrame(X_data).head()

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.10, random_state=42)

multilinear_model.fit(X_train, Y_train)

MultiOutputRegressor(estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...te=42, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
           n_jobs=None)

In [14]:
y_pred = multilinear_model.predict(X_test)
y_pred[1:5]

array([[22.17792327,  0.95986881],
       [29.49383602,  0.33568177],
       [22.39524522,  0.95938102],
       [29.49383602,  0.33568177]])

In [16]:
def predict_multilinear(model: MultiOutputRegressor, encoder: OrdinalEncoder, df: pd.DataFrame):
    return model.predict(encoder.transform(df[['state_name', 'tobacco']]))

In [18]:
predict_multilinear(multilinear_model, fe, pd.DataFrame({'state_name': ['Alaska'],
                                                        'tobacco': ['Pipe']}))

array([[52.27269394,  0.94270034]])